## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_new = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df_new.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df_new['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = []
app_filter = application_df_new['CLASSIFICATION'].value_counts()[application_df_new['CLASSIFICATION'].value_counts() < 528]
application_types_to_replace.append(app_filter.index)  

# Replace in dataframe
for app in application_types_to_replace:
    application_df_new['APPLICATION_TYPE'] = application_df_new['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_new['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
counts = application_df_new['CLASSIFICATION'].value_counts()
counts 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df_new['CLASSIFICATION'].value_counts()[application_df_new['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
cls_filter = application_df_new['CLASSIFICATION'].value_counts()[application_df_new['CLASSIFICATION'].value_counts() < 1883]
classifications_to_replace.append(cls_filter.index.tolist())  

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_new['CLASSIFICATION'] = application_df_new['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_new['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
X = application_df_new.drop(columns='IS_SUCCESSFUL')
X = pd.get_dummies(X)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_new['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.20 )

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=6, activation='sigmoid', input_dim=51))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 312       
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
checkpoint_path = 'checkpoints/weights.{epoch:02d}.hdf5'
callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1, 
                                              period=5,
                                              )
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback])

Epoch 1/100
215/215 [==============================] - 3s 4ms/step - loss: 0.7350 - accuracy: 0.5479
Epoch 2/100
215/215 [==============================] - 1s 3ms/step - loss: 0.6697 - accuracy: 0.5517
Epoch 3/100
215/215 [==============================] - 1s 4ms/step - loss: 0.6440 - accuracy: 0.6489
Epoch 4/100
215/215 [==============================] - 1s 5ms/step - loss: 0.6158 - accuracy: 0.7294
Epoch 5/100
207/215 [===========================>..] - ETA: 0s - loss: 0.5931 - accuracy: 0.7345
Epoch 5: saving model to checkpoints/weights.05.hdf5
215/215 [==============================] - 1s 3ms/step - loss: 0.5930 - accuracy: 0.7342
Epoch 6/100
215/215 [==============================] - 1s 6ms/step - loss: 0.5793 - accuracy: 0.7368
Epoch 7/100
215/215 [==============================] - 1s 4ms/step - loss: 0.5717 - accuracy: 0.7336
Epoch 8/100
215/215 [==============================] - 1s 5ms/step - loss: 0.5677 - accuracy: 0.7357
Epoch 9/100
215/215 [==============================] -

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

858/858 - 1s - loss: 0.5649 - accuracy: 0.7246 - 1s/epoch - 2ms/step
Loss: 0.5649020671844482, Accuracy: 0.7245991230010986


In [16]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')